In [1]:
import scipy, os, sys, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [126]:
study_name = 'Study2_EyeTracking'
date_string = '2020-03-31'
subs = np.arange(151,201)

sim_dir = ('/gpfs_home/jvanbaar/data/jvanbaar/SOC_STRUCT_LEARN/'+
            'ComputationalModel/FeatureRL/Simulations/%s_FeatureRL_CoGrRiNa_%s'%(study_name,date_string))

In [127]:
all_sim_results = pd.DataFrame()
for sub_index in subs:
    print(sub_index, end = ',')
    filenames = glob.glob(sim_dir + '/simulations_subInd-%i_*.csv'%sub_index)
    sims = pd.read_csv(filenames[0],index_col=None, dtype = {'sub_ID':str, 'sub_ind':int})
    
    sims['sub_corr'] = sims['player_choice']==sims['sub_pred']
    sims['model_corr'] = sims['player_choice']==sims['model_pred']
    sims['variant'] = sims['player_type'].apply(lambda x: x[-3:])
    
    sim_results = sims.query('trial < 65').groupby(['variant','sub_ID'],as_index=False).mean()[['sub_ind','sub_ID','variant','sub_corr','model_corr']]
    sim_results = sim_results.rename(columns = {'sub_corr':'sub_acc', 'model_corr':'mean_model_acc'})
    
    model_sim_distribution = sims.groupby(['sim_index','variant'],as_index=False).mean()[['variant','model_corr']]
    for variant in ['nat','inv']:
        model_sim_cond_dat = model_sim_distribution.query('variant == @variant')['model_corr'].describe(percentiles = [0.025, 0.975])
        sim_results.loc[sim_results['variant'] == variant,'2.5%'] = model_sim_cond_dat['2.5%']
        sim_results.loc[sim_results['variant'] == variant,'97.5%'] = model_sim_cond_dat['97.5%']
#         inv_m, inv_ci_l, inv_ci_u = mean_confidence_interval(model_sim_distribution.query('variant == @variant')['model_corr'].values)
#         sim_results.loc[sim_results['variant'] == variant,'95 ci'] = inv_ci_u - inv_m
    
    all_sim_results = all_sim_results.append(sim_results)

151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,

In [128]:
all_sim_results.to_csv(sim_dir + '/all_sim_results.csv')

## Across subjects: compute mean accuracy per dataset, get 95% ci

In [93]:
study_name = 'Study1_Mturk'
date_string = '2020-03-24'
subs = np.arange(1,151)

sim_dir = ('/gpfs_home/jvanbaar/data/jvanbaar/SOC_STRUCT_LEARN/'+
            'ComputationalModel/FeatureRL/Simulations/%s_FeatureRL_CoGrRiNa_%s'%(study_name,date_string))

In [94]:
all_sims = pd.DataFrame()
for sub_index in subs:
    print(sub_index, end = ',')
    filenames = glob.glob(sim_dir + '/simulations_subInd-%i_*.csv'%sub_index)
    sims = pd.read_csv(filenames[0],index_col=None, dtype = {'sub_ID':str, 'sub_ind':int})
    sims['sub_corr'] = sims['player_choice']==sims['sub_pred']
    sims['model_corr'] = sims['player_choice']==sims['model_pred']
    sims['variant'] = sims['player_type'].apply(lambda x: x[-3:])
    all_sims = all_sims.append(sims)

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,

In [95]:
avg_per_sim = all_sims.groupby(['sim_index','variant'],as_index=False).mean()

In [96]:
avg_per_sim_pivot = avg_per_sim.pivot(index = 'sim_index', columns = 'variant', values = 'model_corr').reset_index()

In [97]:
avg_per_sim_pivot.to_csv(sim_dir + '/all_sim_mean_accuracies.csv')